In [ ]:
%load_ext autoreload
%autoreload 2
%pdb on
import pdb

In [ ]:
!pip install "stable-baselines3[extra]>=2.0.0a4"

In [ ]:
![[ -d "reinforcement_learning" ]] && pushd reinforcement_learning; git pull; popd
![[ ! -d "reinforcement_learning" ]] && git clone https://github.com/tianhuil/reinforcement_learning.git
!pushd reinforcement_learning; git show HEAD; popd

In [ ]:
import time
from stable_baselines3 import A2C
from stable_baselines3.common.env_checker import check_env
from reinforcement_learning.src.logistics import Logistics
from stable_baselines3.common.monitor import Monitor

env = Logistics(n_rows=3, n_cols=2, palette_types=1, prob_loading = 0.1, prob_unloading = 0.2, n_steps=100)
check_env(env)
log_dir = "/tmp/gym/{}".format(int(time.time()))
env = Monitor(env, log_dir, allow_early_resets=True)

model = A2C("MultiInputPolicy", env, verbose=1, tensorboard_log="/tmp/tb/").learn(200_000)

In [ ]:
%tensorboard --logdir /tmp/tb

In [ ]:
from stable_baselines3.common.env_util import make_vec_env

# Instantiate the env
vec_env = make_vec_env(Logistics, n_envs=1, env_kwargs=dict())

obs = vec_env.reset()
n_steps = 50
for step in range(n_steps):
    action, _ = model.predict(obs, deterministic=True)
    print(f"Step {step + 1}")
    print("Action: ", action)
    obs, reward, done, info = vec_env.step(action)
    print("reward=", reward, "done=", done)
    vec_env.render()
    if done:
        # Note that the VecEnv resets automatically
        # when a done signal is encountered
        print("Goal reached!", "reward=", reward)
        break